# Language Agent Tree Search

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/llama-index-packs/llama-index-packs-lats/examples/language_agent_tree_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[LATS (Language Agent Tree Search)](https://arxiv.org/pdf/2310.04406.pdf) by Zhou et al. combines LLM capabilities in planning, acting, and reasoning within a Monte Carlo tree search framework, allowing for deliberate and adaptive problem-solving guided by external feedback and self-reflection.

We've implemented this agent as a LlamaPack - you can either pip install it to run it out-of-the-box or call `download_llama_pack` to load the pack.

## Setup

#### Option 1. pip install llama-index-packs-lats

In [ ]:
!pip install llama-index-packs-lats

#### Option 2. `download_llama_pack`

In [ ]:
# from llama_index.core.llama_pack import download_llama_pack

# LATSPacks = download_llama_pack("LATSPack")

### Define Global Settings

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings


llm = OpenAI(model="gpt-3.5-turbo")
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

Settings.llm = llm
Settings.embed_model = embed_model

### Download Data

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-03-22 01:08:53--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-03-22 01:08:53 ERROR 404: Not Found.

--2024-03-22 01:08:53--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-03-22 01:08:54 ERROR 404: Not Found.



In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.core.storage import StorageContext
import os


if not os.path.exists("./storage/lyft"):
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")
else:
    storage_context = StorageContext.from_defaults(persist_dir="./storage/lyft")
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir="./storage/uber")
    uber_index = load_index_from_storage(storage_context)

### Setup Tools

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Setup Agent

Now we can setup the LATS agent.

In [ ]:
from llama_index.core.agent import AgentRunner
from llama_index.packs.lats import LATSAgentWorker


agent_worker = LATSAgentWorker.from_tools(query_engine_tools, llm=llm, verbose=True)
agent = AgentRunner(agent_worker)

## Run Some Queries

In [ ]:
# task = agent.create_task(
#     "Given the revenue growth and risk factors of Uber and Lyft, "
#     "which company is performing better? Please use concrete numbers to inform your decision."
# )

task = agent.create_task(
    "Given the balance sheet and risk factors of Uber and Lyft, "
    "which company is performing better? Please use concrete numbers to inform your decision."
)

In [ ]:
step_output = agent.run_step(task.task_id)

> Selecting node to expand: Observation: Given the balance sheet and risk factors of Uber and Lyft, which company is performing better? Please use concrete numbers to inform your decision.
> Generated new reasoning step: Thought: I need to compare the financial information of Uber and Lyft to determine which company is performing better.
Action: uber_10k
Action Input: {'input': 'balance sheet and risk factors'}
Observation: The company's balance sheet may be impacted by the risks associated with platform users engaging in criminal, violent, or inappropriate activities, leading to safety incidents. These incidents could harm the company's reputation, business, financial condition, and operating results. Additionally, the company may face significant liabilities from safety incidents, inadequate qualification processes, and background checks for platform users. Moreover, the company's financial condition could be affected by potential claims of liability resulting from traffic accidents,

In [ ]:
step_output.task_step.step_state["root_node"].children[0].current_reasoning

[ObservationReasoningStep(observation='Given the balance sheet and risk factors of Uber and Lyft, which company is performing better? Please use concrete numbers to inform your decision.'),
 ActionReasoningStep(thought='I need to compare the financial information of Uber and Lyft to determine which company is performing better.', action='uber_10k', action_input={'input': 'balance sheet and risk factors'}),
 ObservationReasoningStep(observation="The company's balance sheet may be impacted by the risks associated with platform users engaging in criminal, violent, or inappropriate activities, leading to safety incidents. These incidents could harm the company's reputation, business, financial condition, and operating results. Additionally, the company may face significant liabilities from safety incidents, inadequate qualification processes, and background checks for platform users. Moreover, the company's financial condition could be affected by potential claims of liability resulting fr

In [ ]:
step_output = agent.run_step(task.task_id)

> Selecting node to expand: Observation: The company's balance sheet may be impacted by the risks associated with platform users engaging in criminal, violent, or inappropriate activities, leading to safety incidents. These incidents could harm the company's reputation, business, financial condition, and operating results. Additionally, the company may face significant liabilities from safety incidents, inadequate qualification processes, and background checks for platform users. Moreover, the company's financial condition could be affected by potential claims of liability resulting from traffic accidents, injuries, or other incidents involving platform users. Furthermore, the company may need to pay out material amounts due to insurance claims, which could impact its financial condition. The company is also subject to pricing regulations, litigation, and regulatory inquiries that could affect its operating costs and financial results.
> Generated new reasoning step: Thought: I have ga

In [ ]:
step_output.task_step.step_state["root_node"].children[0].children[0].current_reasoning

[ObservationReasoningStep(observation='Given the balance sheet and risk factors of Uber and Lyft, which company is performing better? Please use concrete numbers to inform your decision.'),
 ActionReasoningStep(thought='I need to compare the financial information of Uber and Lyft to determine which company is performing better.', action='uber_10k', action_input={'input': 'balance sheet and risk factors'}),
 ObservationReasoningStep(observation="The company's balance sheet may be impacted by the risks associated with platform users engaging in criminal, violent, or inappropriate activities, leading to safety incidents. These incidents could harm the company's reputation, business, financial condition, and operating results. Additionally, the company may face significant liabilities from safety incidents, inadequate qualification processes, and background checks for platform users. Moreover, the company's financial condition could be affected by potential claims of liability resulting fr

In [ ]:
step_output = agent.run_step(task.task_id)

> Selecting node to expand: Observation: The company's balance sheet may be impacted by the risks associated with platform users engaging in criminal, violent, or inappropriate activities, which could harm the company's reputation and financial condition. Additionally, the company faces risks related to insurance liabilities stemming from traffic accidents or incidents caused by drivers using the platform. Regulatory challenges and potential litigation regarding pricing models could also affect the company's financial results. Furthermore, the company's agreements with drivers and merchants establish enforceable rights and obligations, impacting revenue recognition. The balance sheet may reflect provisions for potential liabilities arising from these risk factors.
> Generated new reasoning step: Thought: The risk factors mentioned can impact both Uber and Lyft's financial performance. I need to gather more specific financial information to compare the two companies.
Action: lyft_10k
Ac

In [ ]:
step_output = agent.run_step(task.task_id)

> Selecting node to expand: Observation: The company's balance sheet may be impacted by various factors such as payment processing charges, hosting and platform-related costs, vehicle lease expenses, personnel-related compensation costs, and depreciation. Additionally, risks to consider include challenges in managing the complexities of expanding their platform, potential negative impacts on revenue in the short term, difficulties in attracting and retaining drivers and businesses for new offerings, regulatory compliance issues, and potential litigation risks related to personal injury and automotive liability.
> Generated new reasoning step: Thought: I have gathered information about the risks associated with Lyft's operations.
Action: lyft_10k
Action Input: {'input': 'balance sheet and risk factors'}
Observation: The company's financial statements likely include information on assets, liabilities, and equity, providing a snapshot of its financial position at a specific point in time.

In [ ]:
step_output = agent.run_step(task.task_id)

> Selecting node to expand: Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Based on the risks mentioned, both Uber and Lyft face challenges related to payment processors, compliance issues, and expansion into new jurisdictions. These risks could impact their financial condition and operations. To determine which company is performing better, we need to compare their revenue growth numbers.
> Generated new reasoning step: Thought: I cannot answer the question with the provided tools.
Answer: I cannot answer the question with the provided tools.
> Generated new reasoning step: Thought: I need to compare the revenue growth of Uber and Lyft to determine which company is performing better.
Action: lyft_10k
Action Input: {'input': 'revenue growth'}
Observation: The provided context information does not contain details related to revenue growth.
> Evaluation for input Given the revenue growth and risk factors of Uber and Lyft, which company 

In [ ]:
response = agent.query(
    "Given the revenue growth and risk factors of Uber and Lyft, "
    "which company is performing better? Please use concrete numbers to inform your decision."
)
print(str(response))